<a href="https://colab.research.google.com/github/XetmanX/Kaggle/blob/main/AIA_image_classification_by_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [170]:
import numpy as np
from keras.datasets import mnist
import matplotlib.pyplot as plt
import tensorflow as tf
import keras

from keras import models,layers,optimizers
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D

np.random.seed(8)
tf.random.set_seed(8)


In [171]:
! wget 'https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/11842/196728/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1609590362&Signature=Ozkd%2BkhwXzPi0%2BAXXwTp%2FN5ITuo2BPTspJn3oKygtcXiVPY7uBnu5Ouwb9eM1gJNVlDEggg47OIzUesQt9rSPQGiL9qhTqzvLxGRWZhAiI1axdpxocdmMWgM61XVy2z9fTk9Q7ICXuoMUGSWLGr2WxG6UQquo5Y7esFTmH2qndJx0txyPpmxiPjUAu5mQ0EZYUpoA8jhq4AjIy7Qyd1mZUUYgtq7WzWFj9Qq0%2FJfxALhSnjKerrfqLf7EnBOKuRfa6Qer8s9sA1iqcDGeYP4fbblEgpfDvLrrDTI24TWobFvBc%2BNN8qTjInhHfZrFlwNT34ZBw1oYov5lsGrj3grEA%3D%3D&response-content-disposition=attachment%3B+filename%3Dwhereami.zip'

The name is too long, 514 chars total.
Trying to shorten...
New name is archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1609590362&Signature=Ozkd+khwXzPi0+AXXwTp%2FN5ITuo2BPTspJn3oKygtcXiVPY7uBnu5Ouwb9eM1gJNVlDEggg47OIzUesQt9rSPQGiL9qhTqzvLxGRWZhAiI1axdpxocdmMWgM61XVy2z9fTk.
--2020-12-30 17:27:03--  https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/11842/196728/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1609590362&Signature=Ozkd%2BkhwXzPi0%2BAXXwTp%2FN5ITuo2BPTspJn3oKygtcXiVPY7uBnu5Ouwb9eM1gJNVlDEggg47OIzUesQt9rSPQGiL9qhTqzvLxGRWZhAiI1axdpxocdmMWgM61XVy2z9fTk9Q7ICXuoMUGSWLGr2WxG6UQquo5Y7esFTmH2qndJx0txyPpmxiPjUAu5mQ0EZYUpoA8jhq4AjIy7Qyd1mZUUYgtq7WzWFj9Qq0%2FJfxALhSnjKerrfqLf7EnBOKuRfa6Qer8s9sA1iqcDGeYP4fbblEgpfDvLrrDTI24TWobFvBc%2BNN8qTjInhHfZrFlwNT34ZBw1oYov5lsGrj3grEA%3D%3D&response-content-disposition=attachment%3B+filename%3Dwhereami.zip
Resolving storage.googleapis.com (storage.googleap

In [172]:

train = tf.keras.preprocessing.image_dataset_from_directory(
    directory='/content/train',
    labels="inferred",
    label_mode="categorical",
    color_mode="rgb",
    subset="training",
    validation_split=0.1,
    batch_size=32,
    image_size=(256, 256),
    seed=8
)

validation = tf.keras.preprocessing.image_dataset_from_directory(
    directory='/content/train',
    labels="inferred",
    label_mode="categorical",
    color_mode="rgb",
    subset="validation",
    validation_split=0.1,
    batch_size=32,
    image_size=(256, 256),
    seed=8
)

Found 2985 files belonging to 15 classes.
Using 2687 files for training.
Found 2985 files belonging to 15 classes.
Using 298 files for validation.


In [173]:
base_model=keras.applications.resnet.ResNet50(include_top=False, weights="imagenet", input_tensor=None, input_shape=(256, 256, 3), pooling='avg')
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [174]:
model = models.Sequential()
model.add(base_model)
model.get_layer('resnet50').trainable = False 
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(15, activation='softmax'))
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 2048)              23587712  
_________________________________________________________________
flatten_8 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_24 (Dense)             (None, 128)               262272    
_________________________________________________________________
dense_25 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_26 (Dense)             (None, 15)                975       
Total params: 23,859,215
Trainable params: 271,503
Non-trainable params: 23,587,712
_________________________________________________________________


In [175]:
model.compile(loss='categorical_crossentropy',optimizer = optimizers.RMSprop(lr=1e-4), metrics=['acc'])
model.fit(train, epochs=10, validation_data=validation)

Epoch 1/10
84/84 [==============================] - 19s 189ms/step - loss: 2.1281 - acc: 0.3567 - val_loss: 1.0342 - val_acc: 0.7819
Epoch 2/10
84/84 [==============================] - 15s 179ms/step - loss: 0.7454 - acc: 0.8355 - val_loss: 0.5680 - val_acc: 0.8322
Epoch 3/10
84/84 [==============================] - 15s 175ms/step - loss: 0.3932 - acc: 0.9066 - val_loss: 0.4045 - val_acc: 0.8725
Epoch 4/10
84/84 [==============================] - 15s 175ms/step - loss: 0.2734 - acc: 0.9364 - val_loss: 0.3544 - val_acc: 0.8859
Epoch 5/10
84/84 [==============================] - 15s 175ms/step - loss: 0.2032 - acc: 0.9453 - val_loss: 0.3140 - val_acc: 0.8993
Epoch 6/10
84/84 [==============================] - 15s 177ms/step - loss: 0.1585 - acc: 0.9637 - val_loss: 0.2873 - val_acc: 0.8826
Epoch 7/10
84/84 [==============================] - 15s 178ms/step - loss: 0.1350 - acc: 0.9703 - val_loss: 0.2969 - val_acc: 0.8893
Epoch 8/10
84/84 [==============================] - 15s 178ms/step - 

In [176]:
test = tf.keras.preprocessing.image.ImageDataGenerator().flow_from_directory(
    directory='/content',
    target_size=(256, 256),
    color_mode="rgb",
    classes=['testset'],
    class_mode="categorical",
    batch_size=32,
    shuffle=False,
    seed=8,
    save_to_dir=None,
    save_prefix="",
    save_format="png",
    follow_links=False,
    subset=None,
    interpolation="nearest",
)

Found 1500 images belonging to 1 classes.


In [177]:
pred = model.predict_generator(test)
pred

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


array([[9.8330951e-05, 2.8621629e-01, 7.7873312e-02, ..., 2.3339928e-05,
        1.9257203e-04, 1.6213772e-03],
       [1.1628323e-05, 7.5028275e-08, 4.0946354e-08, ..., 2.4047697e-06,
        1.8981910e-07, 3.1181509e-07],
       [1.4210799e-02, 5.0907111e-06, 5.0689712e-05, ..., 6.2004574e-06,
        2.3929013e-05, 7.7102021e-03],
       ...,
       [1.7812379e-09, 3.7958980e-06, 1.7157948e-06, ..., 1.2003015e-06,
        1.1413096e-07, 7.3950201e-07],
       [1.5699376e-05, 4.7167279e-07, 2.5682908e-05, ..., 7.1004126e-07,
        1.6265203e-07, 3.0148738e-05],
       [4.3217893e-04, 5.8132795e-05, 2.5761386e-05, ..., 7.1511691e-04,
        2.4332711e-02, 1.7090107e-03]], dtype=float32)

In [178]:
name = ['CALsuburb', 'PARoffice', 'bedroom', 'coast', 'forest', 'highway', 'industrial', 'insidecity', 'kitchen', 'livingroom', 'mountain', 'opencountry', 'store', 'street', 'tallbuilding']
zp = zip(range(0,15), name)
int_to_name = dict(zp)
zp = zip(name, [9,7,12,10,4,14,2,3,0,5,8,6,11,1,13])
name_to_int = dict(zp)

In [179]:
id = list()

for i in test.filenames:
  i = i[8:-4]
  id.append(i)

class_image = list()

for i in pred:
  class_image.append(name_to_int[int_to_name[np.argmax(i)]])

len(class_image)

1500

In [180]:
import pandas as pd
data = {'id':id, 'class':class_image} 
my_submit = pd.DataFrame(data)
my_submit.to_csv('My_submit.csv', index=False)